In [1]:
import optuna
import pandas as pd
import numpy as np
import torch
import json
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score, mean_squared_error
import lightgbm as lgb

In [2]:
msl_90p = pd.read_csv('../input_data/msl_90p.csv')
msl_10p = pd.read_csv('../input_data/msl_10p.csv')

print(len(msl_90p))
print(len(msl_10p))

15083
1565


In [3]:
desc_mol90p = torch.load("../torch_data/train_split/desc_mol90p.pt")
desc_sol90p = torch.load("../torch_data/train_split/desc_sol90p.pt")

desc_mol10p = torch.load("../torch_data/test_split/desc_mol10p.pt")
desc_sol10p = torch.load("../torch_data/test_split/desc_sol10p.pt")

print(desc_mol90p.shape, desc_mol10p.shape, desc_sol90p.shape, desc_sol10p.shape)

C:\Users\kappa\AppData\Local\Temp\ipykernel_22900\852548143.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  desc_mol90p = torch.load("../torch_data/train_split/desc_mol9

(15083, 217) (1565, 217) (15083, 217) (1565, 217)


C:\Users\kappa\AppData\Local\Temp\ipykernel_22900\852548143.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  desc_mol10p = torch.load("../torch_data/test_split/desc_mol10

In [4]:
vecs_mols90p = torch.load("../torch_data/train_split/vecs_mols90p.pt")
vecs_sols90p = torch.load("../torch_data/train_split/vecs_sols90p.pt")

print(vecs_mols90p.shape, vecs_sols90p.shape)

torch.Size([15083, 256]) torch.Size([15083, 256])


C:\Users\kappa\AppData\Local\Temp\ipykernel_22900\2755552372.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vecs_mols90p = torch.load("../torch_data/train_split/vecs_mo

In [5]:
vecs_mols10p = torch.load("../torch_data/test_split/vecs_mols10p.pt")
vecs_sols10p = torch.load("../torch_data/test_split/vecs_sols10p.pt")

print(vecs_mols10p.shape, vecs_sols10p.shape)

torch.Size([1565, 256]) torch.Size([1565, 256])


C:\Users\kappa\AppData\Local\Temp\ipykernel_22900\1807576196.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vecs_mols10p = torch.load("../torch_data/test_split/vecs_mol

In [6]:
chemB_mol90p = torch.load("../torch_data/train_split/chemB_mol90p.pt")
chemB_sol90p = torch.load("../torch_data/train_split/chemB_sol90p.pt")

chemB_mol10p = torch.load("../torch_data/test_split/chemB_mol10p.pt")
chemB_sol10p = torch.load("../torch_data/test_split/chemB_sol10p.pt")

print(chemB_mol90p.shape, chemB_sol90p.shape, chemB_mol10p.shape, chemB_sol10p.shape)

torch.Size([15083, 768]) torch.Size([15083, 768]) torch.Size([1565, 768]) torch.Size([1565, 768])


C:\Users\kappa\AppData\Local\Temp\ipykernel_22900\3944479674.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  chemB_mol90p = torch.load("../torch_data/train_split/chemB_m

In [7]:
molformer_mol90p = torch.load("../torch_data/train_split/molformer_mol90p.pt")
molformer_sol90p = torch.load("../torch_data/train_split/molformer_sol90p.pt")

molformer_mol10p = torch.load("../torch_data/test_split/molformer_mol10p.pt")
molformer_sol10p = torch.load("../torch_data/test_split/molformer_sol10p.pt")

print(molformer_mol90p.shape, molformer_sol90p.shape, molformer_mol10p.shape, molformer_sol10p.shape)

torch.Size([15083, 768]) torch.Size([15083, 768]) torch.Size([1565, 768]) torch.Size([1565, 768])


C:\Users\kappa\AppData\Local\Temp\ipykernel_22900\1912701043.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  molformer_mol90p = torch.load("../torch_data/train_split/mol

In [8]:
fgp_mol90p = torch.load("../torch_data/train_split/fgp_mol90p.pt")
fgp_sol90p = torch.load("../torch_data/train_split/fgp_sol90p.pt")

fgp_mol10p = torch.load("../torch_data/test_split/fgp_mol10p.pt")
fgp_sol10p = torch.load("../torch_data/test_split/fgp_sol10p.pt")

print(fgp_mol90p.shape, fgp_sol90p.shape, fgp_mol10p.shape, fgp_sol10p.shape)

C:\Users\kappa\AppData\Local\Temp\ipykernel_22900\2116207267.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  fgp_mol90p = torch.load("../torch_data/train_split/fgp_mol90

torch.Size([15083, 2048]) torch.Size([15083, 2048]) torch.Size([1565, 2048]) torch.Size([1565, 2048])


C:\Users\kappa\AppData\Local\Temp\ipykernel_22900\2116207267.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  fgp_mol10p = torch.load("../torch_data/test_split/fgp_mol10p

### FP only

In [11]:
X1 = torch.cat([fgp_mol90p, fgp_sol90p], dim=1)
print(X1.shape)

y1 = torch.tensor(msl_90p['Quantum yield'].values)
print(y1.shape)

torch.Size([15083, 4096])
torch.Size([15083])


In [13]:
X_np = X1.cpu().numpy()
y_np = y1.cpu().numpy()

X_train, X_val, y_train, y_val = train_test_split(X_np, y_np, test_size=0.1, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(13574, 4096) (13574,)
(1509, 4096) (1509,)


In [14]:
X_test = torch.cat([fgp_mol10p, fgp_sol10p], dim=1)
y_test = torch.tensor(msl_10p['Quantum yield'].values)

print(X_test.shape, y_test.shape)

torch.Size([1565, 4096]) torch.Size([1565])


In [9]:
with open("../model_params/xgb_params.json", "r") as f:
    params = json.load(f)

In [16]:
xgb_model = XGBRegressor(
    **params,
    n_estimators=2000,
    random_state=42,
    n_jobs=-1,
    tree_method="hist",
    device="cuda"
)

eval_set = [(X_val, y_val)]

xgb_model.fit(X_train, y_train, eval_set=eval_set, verbose=False)

y_test_xgb = xgb_model.predict(X_test)
print("test:")
print(f"R Squared: {r2_score(y_test, y_test_xgb):.3f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_test_xgb)):.3f}")

test:
R Squared: 0.525
RMSE: 0.215


### RDKit only

In [ ]:
desc_mol90p = torch.tensor(desc_mol90p)
desc_sol90p = torch.tensor(desc_sol90p)
desc_mol10p = torch.tensor(desc_mol10p)
desc_sol10p = torch.tensor(desc_sol10p)

In [19]:
X2 = torch.cat([desc_mol90p, desc_sol90p], dim=1)
print(X2.shape)

y2 = torch.tensor(msl_90p['Quantum yield'].values)
print(y2.shape)

torch.Size([15083, 434])
torch.Size([15083])


In [20]:
X_np = X2.cpu().numpy()
y_np = y2.cpu().numpy()

X_train, X_val, y_train, y_val = train_test_split(X_np, y_np, test_size=0.1, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(13574, 434) (13574,)
(1509, 434) (1509,)


In [21]:
X_test = torch.cat([desc_mol10p, desc_sol10p], dim=1)
y_test = torch.tensor(msl_10p['Quantum yield'].values)

print(X_test.shape, y_test.shape)

torch.Size([1565, 434]) torch.Size([1565])


In [22]:
xgb_model = XGBRegressor(
    **params,
    n_estimators=2000,
    random_state=42,
    n_jobs=-1,
    tree_method="hist",
    device="cuda"
)

eval_set = [(X_val, y_val)]

xgb_model.fit(X_train, y_train, eval_set=eval_set, verbose=False)

y_test_xgb = xgb_model.predict(X_test)
print("test:")
print(f"R Squared: {r2_score(y_test, y_test_xgb):.3f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_test_xgb)):.3f}")

test:
R Squared: 0.502
RMSE: 0.221


### FP + RDKit

In [25]:
X3 = torch.cat([fgp_mol90p, fgp_sol90p, desc_mol90p, desc_sol90p], dim=1)
print(X3.shape)

y3 = torch.tensor(msl_90p['Quantum yield'].values)
print(y3.shape)

torch.Size([15083, 4530])
torch.Size([15083])


In [26]:
X_np = X3.cpu().numpy()
y_np = y3.cpu().numpy()

X_train, X_val, y_train, y_val = train_test_split(X_np, y_np, test_size=0.1, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(13574, 4530) (13574,)
(1509, 4530) (1509,)


In [27]:
X_test = torch.cat([fgp_mol10p, fgp_sol10p, desc_mol10p, desc_sol10p], dim=1)
y_test = torch.tensor(msl_10p['Quantum yield'].values)

print(X_test.shape, y_test.shape)

torch.Size([1565, 4530]) torch.Size([1565])


In [28]:
xgb_model = XGBRegressor(
    **params,
    n_estimators=2000,
    random_state=42,
    n_jobs=-1,
    tree_method="hist",
    device="cuda"
)

eval_set = [(X_val, y_val)]

xgb_model.fit(X_train, y_train, eval_set=eval_set, verbose=False)

y_test_xgb = xgb_model.predict(X_test)
print("test:")
print(f"R Squared: {r2_score(y_test, y_test_xgb):.3f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_test_xgb)):.3f}")

test:
R Squared: 0.526
RMSE: 0.215


### COATI only

In [29]:
X4 = torch.cat([vecs_mols90p, vecs_sols90p], dim=1)
print(X4.shape)

y4 = torch.tensor(msl_90p['Quantum yield'].values)
print(y4.shape)

torch.Size([15083, 512])
torch.Size([15083])


In [30]:
X_np = X4.cpu().numpy()
y_np = y4.cpu().numpy()

X_train, X_val, y_train, y_val = train_test_split(X_np, y_np, test_size=0.1, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(13574, 512) (13574,)
(1509, 512) (1509,)


In [31]:
X_test = torch.cat([vecs_mols10p, vecs_sols10p], dim=1)
y_test = torch.tensor(msl_10p['Quantum yield'].values)

print(X_test.shape, y_test.shape)

torch.Size([1565, 512]) torch.Size([1565])


In [32]:
xgb_model = XGBRegressor(
    **params,
    n_estimators=2000,
    random_state=42,
    n_jobs=-1,
    tree_method="hist",
    device="cuda"
)

eval_set = [(X_val, y_val)]

xgb_model.fit(X_train, y_train, eval_set=eval_set, verbose=False)

y_test_xgb = xgb_model.predict(X_test)
print("test:")
print(f"R Squared: {r2_score(y_test, y_test_xgb):.3f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_test_xgb)):.3f}")

test:
R Squared: 0.385
RMSE: 0.245


### ChemBERTa only

In [9]:
X5 = torch.cat([chemB_mol90p, chemB_sol90p], dim=1)
print(X5.shape)

y5 = torch.tensor(msl_90p['Quantum yield'].values)
print(y5.shape)

torch.Size([15083, 1536])
torch.Size([15083])


In [10]:
X_np = X5.cpu().numpy()
y_np = y5.cpu().numpy()

X_train, X_val, y_train, y_val = train_test_split(X_np, y_np, test_size=0.1, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(13574, 1536) (13574,)
(1509, 1536) (1509,)


In [11]:
X_test = torch.cat([chemB_mol10p, chemB_mol10p], dim=1)
y_test = torch.tensor(msl_10p['Quantum yield'].values)

print(X_test.shape, y_test.shape)

torch.Size([1565, 1536]) torch.Size([1565])


In [14]:
xgb_model = XGBRegressor(
    **params,
    n_estimators=2000,
    random_state=42,
    n_jobs=-1,
    tree_method="hist",
    device="cuda"
)

eval_set = [(X_val, y_val)]

xgb_model.fit(X_train, y_train, eval_set=eval_set, verbose=False)

y_test_xgb = xgb_model.predict(X_test)
print("test:")
print(f"R Squared: {r2_score(y_test, y_test_xgb):.3f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_test_xgb)):.3f}")

test:
R Squared: 0.133
RMSE: 0.291


### MolFormer only

In [15]:
X6 = torch.cat([molformer_mol90p, molformer_sol90p], dim=1)
print(X6.shape)

y6 = torch.tensor(msl_90p['Quantum yield'].values)
print(y6.shape)

torch.Size([15083, 1536])
torch.Size([15083])


In [16]:
X_np = X6.cpu().numpy()
y_np = y6.cpu().numpy()

X_train, X_val, y_train, y_val = train_test_split(X_np, y_np, test_size=0.1, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(13574, 1536) (13574,)
(1509, 1536) (1509,)


In [17]:
X_test = torch.cat([molformer_mol10p, molformer_sol10p], dim=1)
y_test = torch.tensor(msl_10p['Quantum yield'].values)

print(X_test.shape, y_test.shape)

torch.Size([1565, 1536]) torch.Size([1565])


In [18]:
xgb_model = XGBRegressor(
    **params,
    n_estimators=2000,
    random_state=42,
    n_jobs=-1,
    tree_method="hist",
    device="cuda"
)

eval_set = [(X_val, y_val)]

xgb_model.fit(X_train, y_train, eval_set=eval_set, verbose=False)

y_test_xgb = xgb_model.predict(X_test)
print("test:")
print(f"R Squared: {r2_score(y_test, y_test_xgb):.3f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_test_xgb)):.3f}")

test:
R Squared: 0.339
RMSE: 0.254


### COATI + FP + RDKit

In [15]:
X7 = torch.cat([vecs_mols90p, vecs_sols90p, fgp_mol90p, fgp_sol90p, desc_mol90p, desc_sol90p], dim=1)
print(X7.shape)

y7 = torch.tensor(msl_90p['Quantum yield'].values)
print(y7.shape)

torch.Size([15083, 5042])
torch.Size([15083])


In [16]:
X_np = X7.cpu().numpy()
y_np = y7.cpu().numpy()

X_train, X_val, y_train, y_val = train_test_split(X_np, y_np, test_size=0.1, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(13574, 5042) (13574,)
(1509, 5042) (1509,)


In [17]:
X_test = torch.cat([vecs_mols10p, vecs_sols10p, fgp_mol10p, fgp_sol10p, desc_mol10p, desc_sol10p], dim=1)
y_test = torch.tensor(msl_10p['Quantum yield'].values)

print(X_test.shape, y_test.shape)

torch.Size([1565, 5042]) torch.Size([1565])


In [18]:
xgb_model = XGBRegressor(
    **params,
    n_estimators=2000,
    random_state=42,
    n_jobs=-1,
    tree_method="hist",
    device="cuda"
)

eval_set = [(X_val, y_val)]

xgb_model.fit(X_train, y_train, eval_set=eval_set, verbose=False)

y_test_xgb = xgb_model.predict(X_test)
print("test:")
print(f"R Squared: {r2_score(y_test, y_test_xgb):.3f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_test_xgb)):.3f}")

XGBoostError: [17:46:45] C:\actions-runner\_work\xgboost\xgboost\src\common\device_vector.cu:23: Memory allocation error on worker 0: bad allocation: cudaErrorMemoryAllocation: out of memory
- Free memory: 4.36914GB
- Requested memory: 1GB


### ChemBERTa + FP + RDKit

In [11]:
X8 = torch.cat([chemB_mol90p, chemB_sol90p, fgp_mol90p, fgp_sol90p, desc_mol90p, desc_sol90p], dim=1)
print(X8.shape)

y8 = torch.tensor(msl_90p['Quantum yield'].values)
print(y8.shape)

torch.Size([15083, 6066])
torch.Size([15083])


In [12]:
X_np = X8.cpu().numpy()
y_np = y8.cpu().numpy()

X_train, X_val, y_train, y_val = train_test_split(X_np, y_np, test_size=0.1, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(13574, 6066) (13574,)
(1509, 6066) (1509,)


In [13]:
X_test = torch.cat([chemB_mol10p, chemB_sol10p, fgp_mol10p, fgp_sol10p, desc_mol10p, desc_sol10p], dim=1)
y_test = torch.tensor(msl_10p['Quantum yield'].values)

print(X_test.shape, y_test.shape)

torch.Size([1565, 6066]) torch.Size([1565])


In [14]:
xgb_model = XGBRegressor(
    **params,
    n_estimators=2000,
    random_state=42,
    n_jobs=-1,
    tree_method="hist",
    device="cuda"
)

eval_set = [(X_val, y_val)]

xgb_model.fit(X_train, y_train, eval_set=eval_set, verbose=False)

y_test_xgb = xgb_model.predict(X_test)
print("test:")
print(f"R Squared: {r2_score(y_test, y_test_xgb):.3f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_test_xgb)):.3f}")

test:
R Squared: 0.411
RMSE: 0.240


### MolFormer + FP + RDKIT

In [15]:
X9 = torch.cat([molformer_mol90p, molformer_sol90p, fgp_mol90p, fgp_sol90p, desc_mol90p, desc_sol90p], dim=1)
print(X9.shape)

y9 = torch.tensor(msl_90p['Quantum yield'].values)
print(y9.shape)

torch.Size([15083, 6066])
torch.Size([15083])


In [16]:
X_np = X9.cpu().numpy()
y_np = y9.cpu().numpy()

X_train, X_val, y_train, y_val = train_test_split(X_np, y_np, test_size=0.1, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(13574, 6066) (13574,)
(1509, 6066) (1509,)


In [17]:
X_test = torch.cat([molformer_mol10p, molformer_sol10p, fgp_mol10p, fgp_sol10p, desc_mol10p, desc_sol10p], dim=1)
y_test = torch.tensor(msl_10p['Quantum yield'].values)

print(X_test.shape, y_test.shape)

torch.Size([1565, 6066]) torch.Size([1565])


In [18]:
xgb_model = XGBRegressor(
    **params,
    n_estimators=2000,
    random_state=42,
    n_jobs=-1,
    tree_method="hist",
    device="cuda"
)

eval_set = [(X_val, y_val)]

xgb_model.fit(X_train, y_train, eval_set=eval_set, verbose=False)

y_test_xgb = xgb_model.predict(X_test)
print("test:")
print(f"R Squared: {r2_score(y_test, y_test_xgb):.3f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_test_xgb)):.3f}")

test:
R Squared: 0.412
RMSE: 0.240
